In [1]:
from arguments import DataArguments
from transformers import AutoConfig, AutoTokenizer
import torch

In [2]:
data_args = DataArguments(train_data = "/home/ubuntu/mosaic-embedding-pairs", train_group_size = 3,
             query_max_len = 8192, passage_max_len = 8192)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama-600m-hf-32768-fpf')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

In [8]:
from data_contrastive import TrainDatasetForEmbedding, EmbedCollator

In [9]:
dataset = TrainDatasetForEmbedding(data_args, tokenizer)

In [14]:
b = [dataset[i] for i in range(2)]
b

[(['Pelaburan Syarikat China di Malaysia Tingkat Hubungan Dua Hala\n\nUntuk mendapatkan maklumat terkini, ikuti kami melalui Telegram\nKuala Lumpur –\xa0 Menteri Perdagangan Antarabangsa dan Industri, Tengku Datuk Seri Utama Tengku Zafrul Aziz berkata peningkatan minat syarikat China melabur di Malaysia memberi petanda baik kepada negara dan telah meningkatkan hubungan dua hala antara Malaysia dan China serta telah disokong oleh keyakinan terhadap kerajaan Perpaduan negara.\nBeliau berkata menerusi satu kenyataan yang dikeluarkan oleh Lembaga Pembangunan Pelaburan Malaysia (MIDA), Kementerian Perdagangan Antarabangsa dan Industri\xa0 (MITI) akan terus membantu memudahkan urusan para pelabur untuk menjalankan perniagaan di negara ini bagi menunjukkan bahawa Malaysia adalah sebuah negara sentiasa menyokong industri, perdagangan. Beliau juga menzahirkan ucapan tahniah kepada MIDA dan semua agensi berkaitan dalam membantu mendapatkan pelaburan berpotensi yang bernilai RM 170 bilion. “MITI,

In [15]:
collactor = EmbedCollator(
            tokenizer,
            query_max_len=data_args.query_max_len,
            passage_max_len=data_args.passage_max_len
        )

In [16]:
b_ = collactor(b)
b_

{'query': {'input_ids': tensor([[    1, 15549,   370,  ...,  3480,  2518, 29889],
         [    1, 15549,   370,  ...,  3480,  2518, 29889],
         [    1, 15549,   370,  ...,  3480,  2518, 29889],
         ...,
         [    1,   317,   423,  ...,     0,     0,     0],
         [    1,   317,   423,  ...,     0,     0,     0],
         [    1,   317,   423,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'passage': {'input_ids': tensor([[    1,   317,   423,  ...,     0,     0,     0],
         [    1,  6225,   557,  ...,     0,     0,     0],
         [    1,  6225,   557,  ...,     0,     0,     0],
         ...,
         [    1,   383,   801,  ..., 29885,  5832, 29889],
         [    1,   383,   801,  ..., 29885,  5832, 29889],
         [    1, 15549,   370

In [17]:
torch.tensor(b_['label'])

tensor([1, 0, 1, 0, 1, 0, 0, 1])

In [18]:
from modeling import LlamaModelEmbedding
from transformers import LlamaConfig

config = LlamaConfig.from_pretrained(
    'mesolitica/llama-600m-hf-32768-fpf',
    num_labels=1,
)

config.temperature = 0.02
config.normalized = True
config.sentence_pooling_method = 'mean'

In [19]:
model = LlamaModelEmbedding.from_pretrained('mesolitica/llama-600m-hf-32768-fpf', config = config)

Some weights of LlamaModelEmbedding were not initialized from the model checkpoint at mesolitica/llama-600m-hf-32768-fpf and are newly initialized: ['model.dense_layer.bias', 'model.dense_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
l = model.encode(b_['query'])

In [21]:
r = model.encode(b_['passage'])

In [22]:
import torch.nn.functional as F
COSINE_DISTANCE = lambda x, y: 1-F.cosine_similarity(x, y)

In [23]:
distances = COSINE_DISTANCE(l, r)
distances

tensor([0.4240, 0.3242, 0.2538, 0.3378, 0.4240, 0.4673, 0.4673, 0.4240],
       grad_fn=<RsubBackward1>)

In [42]:
distances = labels.float()

In [43]:
distances, labels

(tensor([1., 0., 1., 0., 1., 0., 0., 1.]), tensor([1, 0, 1, 0, 1, 0, 0, 1]))

In [44]:
labels = torch.tensor(b_['label'])
margin = 0.5

In [45]:
losses = 0.5 * (labels.float() * distances.pow(2) + (1 - labels).float() * F.relu(margin - distances).pow(2))
losses

tensor([0.5000, 0.1250, 0.5000, 0.1250, 0.5000, 0.1250, 0.1250, 0.5000])

In [27]:
(1 - labels).float() * F.relu(margin - distances).pow(2)

tensor([0.0000, 0.0309, 0.0000, 0.0263, 0.0000, 0.0011, 0.0011, 0.0000],
       grad_fn=<MulBackward0>)

In [28]:
(1 - labels).float()

tensor([0., 1., 0., 1., 0., 1., 1., 0.])

In [29]:
 F.relu(margin - distances).pow(2)

tensor([0.0058, 0.0309, 0.0606, 0.0263, 0.0058, 0.0011, 0.0011, 0.0058],
       grad_fn=<PowBackward0>)

In [31]:
(labels.float() * distances.pow(2))

tensor([0.1798, 0.0000, 0.0644, 0.0000, 0.1798, 0.0000, 0.0000, 0.1798],
       grad_fn=<MulBackward0>)